In [1]:
import pandas as pd
import numpy as np 
from dbfread import DBF
import psycopg2

# Colunas de dados necessario para criar o banco de dados INNOVARO

## colunas:

1- Código <br>
2- Nome <br>
3- classe <br>
4- Classificação fiscal <br>
5- UN <br>
6- Procedência <br>
7- Fabricante <br>
8- Referência fabricante <br>
9- Marca <br>
10- Modelo <br>
11- Coleção/Família <br>
12- Cor <br>
13- Tamanho <br>
14- Quantidade P/Volume <br>
15- Quantidade P/Vol Compra <br>
16- Data em que o produto saiu de linha <br>
17- Informador <br>
18- Data em que o produto tornou-se inativo <br>
19- inativador <br>
20- Material <br>
21- Referência principal <br>
22- Descriçao Genérica <br>
23- Cest <br>
24- Tipo de Medicamento <br>
25- Outras Caracteristicas <br>
26- Validade(meses)

## upload SDB Principal 

In [2]:
estoq001 = DBF('ESTOQ001.dbf', encoding='latin1')
estoq001 = pd.DataFrame(estoq001)

## visualização do banco de dados

In [3]:
estoq001.head(3)

,TQCLASSI,TQCODIGO,TQGRADE,TQDESCRI,TQCODBAR,TQEMBENT,TQENTQTD,TQEMBSAI,TQSAIQTD,TQQTDMAX,...,TQIDPROM,TQQTDPRO,TQVALPRO,TQRASTRO,TQOPERAD,TQULTPRE,TQMARCA,TQBENEFI,TQDESONE,TQALIFCP
0,10.033.052,1,UN,TORQUES ARMADOR,,UN,1,UN,1,0,...,0,0.0,0.0,,37,11.16,,,,0.0
1,10.019.022,2,UN,ACESSORIO VERAO,,UN,1,UN,1,0,...,0,0.0,0.0,,37,5.16,,,,0.0
2,10.033.052,3,UN,X 8EQUOT,,UN,1,UN,1,0,...,0,0.0,0.0,,37,0.00,,,,0.0


## identificação de informações das colunas do dataframe

In [4]:
estoq001.info(verbose = True, show_counts= True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86475 entries, 0 to 86474
Data columns (total 102 columns):
 #    Column     Non-Null Count  Dtype  
---   ------     --------------  -----  
 0    TQCLASSI   86475 non-null  object 
 1    TQCODIGO   86475 non-null  object 
 2    TQGRADE    86475 non-null  object 
 3    TQDESCRI   86475 non-null  object 
 4    TQCODBAR   86475 non-null  object 
 5    TQEMBENT   86475 non-null  object 
 6    TQENTQTD   86475 non-null  int64  
 7    TQEMBSAI   86475 non-null  object 
 8    TQSAIQTD   86475 non-null  int64  
 9    TQQTDMAX   86475 non-null  int64  
 10   TQQTDMIN   86475 non-null  int64  
 11   TQSALDO    85689 non-null  float64
 12   TQULTENT   8417 non-null   object 
 13   TQULTSAI   27694 non-null  object 
 14   TQPREVIS   0 non-null      object 
 15   TQPRECUS   86475 non-null  float64
 16   TQPREVEN   86475 non-null  float64
 17   TQSUBSTI   86475 non-null  float64
 18   TQTRIBUT   86475 non-null  object 
 19   TQALIQUO   86475 non-nu

## seleção de colunas para o banco de dados do INNOVARO 

deverá ser inserido apenas as colunas que queremos visualizar no banco de dados do Prosuper

In [5]:
colunas = ['TQCODIGO', 'TQDESCRI', 'TQGRADE', 'TQOBSERV', 'TQPRECUS', 'TQPREVEN', 'TQNCM', 'TQCFOP', 'TQCEST', 'TQGRUPO', 'TQSGRUPO']

## filtrando as colunas no banco de dados

In [6]:
estoq001_filtro = estoq001[colunas]

In [7]:
estoq001_filtro

,TQCODIGO,TQDESCRI,TQGRADE,TQOBSERV,TQPRECUS,TQPREVEN,TQNCM,TQCFOP,TQCEST,TQGRUPO,TQSGRUPO
0,1,TORQUES ARMADOR,UN,TORQUES ARMADOR,0.000,9.99,8203.20.10,5405,0800800,LEDS,LEDS/LED
1,2,ACESSORIO VERAO,UN,ACESSORIO VERAO,0.000,0.00,8205.40.00,5102,,BIJUTERIAS,BIJUTERIA/ACESSORIOS VERAO
2,3,X 8EQUOT,UN,CHAVE DE FENDA 1/4EQUOT,0.000,5.80,8205.40.00,5405,,LEDS,LEDS/LED
3,4,CHAVE PHILLIPS 1/4,UN,CHAVE PHILLIPS 1/4EQUOT,0.000,5.16,8205.40.00,5405,,LEDS,LEDS/FERRAMENTA
4,5,X 8EQUOT,UN,CHAVE PHILLIPS 1/4EQUOT,0.000,5.80,8205.40.00,5405,,LEDS,LEDS/FERRAMENTA
...,...,...,...,...,...,...,...,...,...,...,...
86470,91691,TELA PARA PINTURA,UN,TELA PARA PINTURA 20X20,1.136,0.00,4414.10.00,5102,,PAPELARIA,PAPELARIA/PAPELARIA
86471,91692,TELA PARA PINTURA,UN,TELA PARA PINTURA 24X30,1.563,0.00,4414.10.00,5102,,PAPELARIA,PAPELARIA/PAPELARIA
86472,91693,TELA PARA PINTURA,UN,TELA PARA PINTURA,2.160,0.00,4414.10.00,5102,,PAPELARIA,PAPELARIA/PAPELARIA
86473,91694,KINDER OVO JOY,UN,CHOC KINDER OVO JOY,0.000,0.00,1806.90.00,5102,1700400,CONVENIENCIA,CONVENIENCIA/ALIMENTOS


## identificando valores do dataframe estoq001_filtro

In [8]:
estoq001_filtro.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86475 entries, 0 to 86474
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   TQCODIGO  86475 non-null  object 
 1   TQDESCRI  86475 non-null  object 
 2   TQGRADE   86475 non-null  object 
 3   TQOBSERV  86475 non-null  object 
 4   TQPRECUS  86475 non-null  float64
 5   TQPREVEN  86475 non-null  float64
 6   TQNCM     86475 non-null  object 
 7   TQCFOP    86475 non-null  object 
 8   TQCEST    86475 non-null  object 
 9   TQGRUPO   86475 non-null  object 
 10  TQSGRUPO  86475 non-null  object 
dtypes: float64(2), object(9)
memory usage: 7.3+ MB


## renomeando colunas do dataframe

In [9]:
estoq001_filtro.rename({'TQCODIGO':'CODIGO', 'TQDESCRI':'DESCRICAO', 'TQGRADE' : 'GRADE', 'TQOBSERV':'OBSERVAÇÃO/DESCRIÇÃO COMPLETA', 'TQPRECUS':'CUSTO', 'TQPREVEN':'VENDA', 'TQNCM':'NCM', 'TQCFOP': 'CFOP',
                           'TQCEST':'CEST', 'TQGRUPO':'SETOR', 'TQSGRUPO': 'SUB-GRUPO'}, axis = 1 , inplace = True)

C:\Users\murillo\AppData\Local\Temp\ipykernel_10500\194538697.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  estoq001_filtro.rename({'TQCODIGO':'CODIGO', 'TQDESCRI':'DESCRICAO', 'TQGRADE' : 'GRADE', 'TQOBSERV':'OBSERVAÇÃO/DESCRIÇÃO COMPLETA', 'TQPRECUS':'CUSTO', 'TQPREVEN':'VENDA', 'TQNCM':'NCM', 'TQCFOP': 'CFOP',


# UPLOAD BANCO DE DADOS COM CODIGOS DE BARRAS

In [11]:
escodbar = DBF('ESCODBAR.dbf', encoding= 'latin1')
escodbar = pd.DataFrame(escodbar)

## visualizando o dataframe de codigos de barras 

In [12]:
escodbar.head(3)

,CBCODBAR,CBCLASSI,CBCODIGO,CBGRADE,CBDESCRI,CBNOMANT,CBID,CBDUPLIC,CBVASILH
0,93,10.033.052,35252,UN,92 PAR20 7W B/Q,LAMPADA PAR 20 7W 3500K,1,P,
1,355,10.020.022,11463,UN,VIOLINHA COLORIDA DE,VIOLINHA COLORIDA DE PLASTICO,2,P,
2,356,10.020.022,11464,UN,GUITARRINHA COLORIDA,GUITARRINHA COLORIDA DE PLASTICO,3,P,


# alterando o nome das colunas do dataframe

In [13]:
escodbar.rename({'CBCODBAR': 'EAN', 'CBCLASSI':'CLASSIFICAÇÃO', 'CBCODIGO': 'CODIGO', 'CBGRADE': 'GRADE', 'CBDESCRI':'DESCRIÇÃO', 'CBNOMANT':'OBSERVAÇÃO/DESCRIÇÃO RESUMIDA',
                    'CBID':'ID' , 'CBDUPLIC': 'DUPLICATAS', 'CBVASILH': 'VASILHAME'}, axis = 1, inplace= True)

## VISUALIZANDO DADOS DO DATAFRAME

In [14]:
escodbar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143045 entries, 0 to 143044
Data columns (total 9 columns):
 #   Column                         Non-Null Count   Dtype 
---  ------                         --------------   ----- 
 0   EAN                            143045 non-null  object
 1   CLASSIFICAÇÃO                  143045 non-null  object
 2   CODIGO                         143045 non-null  object
 3   GRADE                          143045 non-null  object
 4   DESCRIÇÃO                      143045 non-null  object
 5   OBSERVAÇÃO/DESCRIÇÃO RESUMIDA  143045 non-null  object
 6   ID                             143045 non-null  int64 
 7   DUPLICATAS                     143045 non-null  object
 8   VASILHAME                      143045 non-null  object
dtypes: int64(1), object(8)
memory usage: 9.8+ MB


# juntando dados finais

In [15]:
bancoFinal = pd.merge(estoq001_filtro, escodbar , on = 'CODIGO', how= 'inner')

# visualizando o banco de dados final

In [16]:
bancoFinal.head(25)

,CODIGO,DESCRICAO,GRADE_x,OBSERVAÇÃO/DESCRIÇÃO COMPLETA,CUSTO,VENDA,NCM,CFOP,CEST,SETOR,SUB-GRUPO,EAN,CLASSIFICAÇÃO,GRADE_y,DESCRIÇÃO,OBSERVAÇÃO/DESCRIÇÃO RESUMIDA,ID,DUPLICATAS,VASILHAME
0,1,TORQUES ARMADOR,UN,TORQUES ARMADOR,0.0,9.99,8203.20.10,5405,0800800,LEDS,LEDS/LED,7899842205871,10.033.052,UN,TORQUES ARMADOR,TORQUES ARMADOR,143505,R,
1,1,TORQUES ARMADOR,UN,TORQUES ARMADOR,0.0,9.99,8203.20.10,5405,0800800,LEDS,LEDS/LED,179874,10.033.052,UN,TORQUES ARMADOR,TORQUES ARMADOR,143530,R,
2,2,ACESSORIO VERAO,UN,ACESSORIO VERAO,0.0,0.00,8205.40.00,5102,,BIJUTERIAS,BIJUTERIA/ACESSORIOS VERAO,34454,10.019.022,UN,ACESSORIO VERAO,ACESSORIO VERAO,102751,R,
3,3,X 8EQUOT,UN,CHAVE DE FENDA 1/4EQUOT,0.0,5.80,8205.40.00,5405,,LEDS,LEDS/LED,7897373164162,10.033.052,UN,X 8EQUOT,CHAVE DE FENDA 1/4EQUOT,31437,T,
4,3,X 8EQUOT,UN,CHAVE DE FENDA 1/4EQUOT,0.0,5.80,8205.40.00,5405,,LEDS,LEDS/LED,34455,10.033.052,UN,X 8EQUOT,CHAVE DE FENDA 1/4EQUOT,102752,P,
5,4,CHAVE PHILLIPS 1/4,UN,CHAVE PHILLIPS 1/4EQUOT,0.0,5.16,8205.40.00,5405,,LEDS,LEDS/FERRAMENTA,7897373164100,10.033.050,UN,X 5EQUOT,CHAVE PHILLIPS 1/4EQUOT,31434,T,
6,4,CHAVE PHILLIPS 1/4,UN,CHAVE PHILLIPS 1/4EQUOT,0.0,5.16,8205.40.00,5405,,LEDS,LEDS/FERRAMENTA,34457,10.001.002,UN,CHAVE PHILLIPS 1/4,CHAVE PHILLIPS 1/4EQUOT,102754,P,
7,5,X 8EQUOT,UN,CHAVE PHILLIPS 1/4EQUOT,0.0,5.80,8205.40.00,5405,,LEDS,LEDS/FERRAMENTA,7897373164124,10.033.050,UN,X 8EQUOT,CHAVE PHILLIPS 1/4EQUOT,31435,T,
8,5,X 8EQUOT,UN,CHAVE PHILLIPS 1/4EQUOT,0.0,5.80,8205.40.00,5405,,LEDS,LEDS/FERRAMENTA,7890000012430,10.033.050,UN,X 8EQUOT,CHAVE PHILLIPS 1/4EQUOT,139840,T,
9,5,X 8EQUOT,UN,CHAVE PHILLIPS 1/4EQUOT,0.0,5.80,8205.40.00,5405,,LEDS,LEDS/FERRAMENTA,374316,10.033.050,UN,X 8EQUOT,CHAVE PHILLIPS 1/4EQUOT,143532,T,


In [17]:
bancoFinal.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 168827 entries, 0 to 168826
Data columns (total 19 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   CODIGO                         168827 non-null  object 
 1   DESCRICAO                      168827 non-null  object 
 2   GRADE_x                        168827 non-null  object 
 3   OBSERVAÇÃO/DESCRIÇÃO COMPLETA  168827 non-null  object 
 4   CUSTO                          168827 non-null  float64
 5   VENDA                          168827 non-null  float64
 6   NCM                            168827 non-null  object 
 7   CFOP                           168827 non-null  object 
 8   CEST                           168827 non-null  object 
 9   SETOR                          168827 non-null  object 
 10  SUB-GRUPO                      168827 non-null  object 
 11  EAN                            168827 non-null  object 
 12  CLASSIFICAÇÃO                 

In [19]:
artsystem = pd.read_json('sdb/maindata_sdbartsystem.json')
cdm = pd.read_json('sdb/maindata_nfedatainformations.json')

C:\Users\murillo\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\dtypes\astype.py:189: RuntimeWarning: invalid value encountered in cast
  return values.astype(dtype, copy=copy)


In [23]:
artsystem.head(3)

,plu,descriptions,ncm,ean,department,cost,cost_and_taxes,sale,suppliers_code,suppliers
0,375412,PANTUFA INFANTIL FEMININA,64041900,537298512071,BIJUTERIASACESS FRIO,22.6122,22.6122,44.99,TX1207AV,
1,375429,PANTUFA INFANTIL FEMININA,64041900,537298712778,BIJUTERIASACESS FRIO,25.3422,25.3422,50.49,TX1277CH,
2,375498,PANTUFA INFANTIL FEMININA,64041900,537298712075,BIJUTERIASACESS FRIO,21.1302,21.1302,42.49,TX1207GA,


In [24]:
cdm.head(3)

,id,reference,ean,eantributed,description,amount,realcost,costcalculate,unitvalue,totalvalue,...,coffinsvalue,aliqfrete,fretevalue,nfenum,fornecedor,cnpjforn,lojaentrada,cnpjlojaentrada,keynfe,frete_values
0,300200001417896058500711AMENDOIM 70G CONF COLO...,3002000014,17896058500711,7896058500714,AMENDOIM 70G CONF COLORIDO DORI (CX 30UN)(FD10...,2.0,56.99,56.99,56.99,113.98,...,7.10,0,0,267167,JJGG-DISTRIBUIDORA ASSIS ALIMENTOS LTDA,9553327000165,CASA DA MAMAE LOJA DE DEP VAR ATAC EIRELI,34408066000137,3.522121e+43,0.0
1,35010024167891151040693B PIRUL POP MANIA MAXXI...,3501002416,7891151040693,17891151040690,B PIRUL POP MANIA MAXXI LUCCAS NETO,1.0,12.69,12.69,12.69,12.69,...,0.79,0,0,267167,JJGG-DISTRIBUIDORA ASSIS ALIMENTOS LTDA,9553327000165,CASA DA MAMAE LOJA DE DEP VAR ATAC EIRELI,34408066000137,3.522121e+43,0.0
2,35010024117891151040983B PIRUL POP MANIA MAXXI...,3501002411,7891151040983,7891151040983,B PIRUL POP MANIA MAXXI MELANC 24UN,2.0,12.69,12.69,12.69,25.38,...,1.58,0,0,267167,JJGG-DISTRIBUIDORA ASSIS ALIMENTOS LTDA,9553327000165,CASA DA MAMAE LOJA DE DEP VAR ATAC EIRELI,34408066000137,3.522121e+43,0.0


In [50]:
plu = []
for n in artsystem['plu']:
    plu.append(n)
plu

[375412,
 375429,
 375498,
 121071,
 35080,
 114417,
 110433,
 110440,
 306249,
 103350,
 33666,
 32484,
 399760,
 20593,
 297271,
 297288,
 169172,
 169189,
 169196,
 185134,
 109598,
 135894,
 109505,
 109574,
 458917,
 185141,
 109628,
 109611,
 135887,
 135818,
 135825,
 109581,
 109512,
 109604,
 129121,
 135795,
 136013,
 136020,
 135924,
 135931,
 135948,
 135979,
 135986,
 135993,
 135801,
 135832,
 135849,
 135856,
 135863,
 135870,
 210621,
 136037,
 229067,
 163743,
 163750,
 163767,
 109499,
 109635,
 135900,
 135917,
 109567,
 35085,
 35083,
 336130,
 109314,
 377515,
 377478,
 377454,
 190251,
 190275,
 190282,
 337021,
 33682,
 226509,
 226431,
 226448,
 33677,
 292870,
 104555,
 121958,
 33691,
 284912,
 34906,
 466776,
 466769,
 205085,
 35235,
 102261,
 337144,
 337137,
 454063,
 454575,
 338059,
 140157,
 109307,
 110662,
 326391,
 326353,
 326377,
 173780,
 173773,
 123891,
 123914,
 338066,
 176712,
 162036,
 199643,
 110655,
 133678,
 33669,
 123488,
 123761,
 123

In [55]:
%%time
bancoFinal['status'] = bancoFinal['EAN'].apply(lambda x: True if x in plu else False)
list(bancoFinal['status'].drop_duplicates())

CPU times: total: 46.9 ms
Wall time: 56.2 ms


[False]

In [53]:
%%time
selection = artsystem['plu'] == '230223'
artsystem[selection]

CPU times: total: 31.2 ms
Wall time: 2.01 ms


,plu,descriptions,ncm,ean,department,cost,cost_and_taxes,sale,suppliers_code,suppliers


In [47]:
selection = bancoFinal['status'] == True
bancoFinal[selection]

,CODIGO,DESCRICAO,GRADE_x,OBSERVAÇÃO/DESCRIÇÃO COMPLETA,CUSTO,VENDA,NCM,CFOP,CEST,SETOR,SUB-GRUPO,EAN,CLASSIFICAÇÃO,GRADE_y,DESCRIÇÃO,OBSERVAÇÃO/DESCRIÇÃO RESUMIDA,ID,DUPLICATAS,VASILHAME,status


In [118]:
selection = query['VENDA'].str.contains('.49', regex=True)
valores49 = query[selection]